In [163]:
# Model design
import agentpy as ap

MOVES = [(1,0), [0,1], (-1, 0), (0,-1)]

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import random

In [167]:
class dirtyRoomModel(ap.Model):

    def setup(self):
        robots = int(self.p['robot'])
        self.cleaningBot =  ap.AgentList(self, robots)
        self.dirt = ap.AgentList(self, int(self.p['Dirty Rooms Density'] * (self.p.N * self.p.M)))

        self.dirtyRoom = ap.Grid(self, [self.p.N, self.p.M], track_empty=True)
        self.dirtyRoom.add_agents(self.cleaningBot, [(1,1)]*robots)
        self.dirtyRoom.add_agents(self.dirt, random = True)

        

        # Create dirt (Rooms)
        #n_rooms = int(self.p['Dirty Rooms Density'] * (self.p.N * self.p.M))
        
        #rooms = self.dirt = ap.AgentList(self, n_rooms)

        # Create grid (Rooms)
        # self.dirtyRoom = ap.Grid(self, [self.p.N, self.p.M ], track_empty=True)
        # self.dirtyRoom.add_agents(rooms, random=True, empty=True)
        # self.dirtyRoom.add_agents(self.cleaningBot,[(1,1)])


        # Initiate a dynamic variable for all rooms
        # Condition 0: cleaningBot, 1: Cleaning, 2: Cleaned
        # self.cleaningBot.is_cleaningBot = True
        # self.dirt.is_cleaningBot = False
    
        # self.dirt.is_clean= False

        self.cleaningBot.type_agent = 0
        self.dirt.type_agent = 1
        # Start the cleaning from the left side of the grid

    def step(self):

        # Select burning n_rooms
        dirtRoom = self.dirt.select(self.dirt.type_agent == 1)
        cleaningBots = self.cleaningBot
        # Spread fire
        for cleaningBot in cleaningBots:
            for neighbor in self.dirtyRoom.neighbors(cleaningBot):
                if neighbor.type_agent == 1:
                    neighbor.type_agent = 2 # Neighbor starts burning
                    break 
            else:
                self.dirtyRoom.move_by(cleaningBot, random.choice(MOVES))
            
        dirt_pending = dirtRoom
        # Stop simulation if no fire is left
        if len(dirt_pending) == 0:
            self.stop()

    def end(self):

        # Document a measure at the end of the simulation
        dirtyRoom = len(self.dirt.select(self.dirt.type_agent == 2))
        self.report('Percentage of dirty rooms',
                    dirtyRoom / len(self.dirt))

In [165]:
# Define parameters

parameters = {
    'Dirty Rooms Density': 0.8, # Percentage of grid covered by dirt
    'N': 30, # Height and length of the grid
    'M': 80,
    'steps': 100,
    'robot' : 30,
}

In [168]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    initial = int(model.p['Dirty Rooms Density'] * (model.p.N * model.p.M))
    cleanedRooms = float("{:.2f}".format(100 - ((len(model.dirt.select(model.dirt.type_agent == 1)) * 100) / initial)))
    attr_grid = model.dirtyRoom.attr_grid('type_agent')
    color_dict = {0:'#7FC97F', 1:'#d62c2c', 2:'#e5e5e5', None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.tick_params(axis='x', colors='white')    
    ax.tick_params(axis='y', colors='white')
    ax.set_title(f"Simulation of a cleaning robot\n"
                 f"Time-step: {model.t}, Rooms left: "
                 f"{len(model.dirt.select(model.dirt.type_agent == 1))}\n" 
                 f"Percentage of clean rooms: "
                 f" {cleanedRooms}%" , color= 'white')


fig, ax = plt.subplots()
model = dirtyRoomModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))